In [1]:
import pandas as pd
import numpy as np

In [2]:
links=pd.read_csv('links.csv')
movies=pd.read_csv('movies.csv')
ratings=pd.read_csv('ratings.csv')
tags=pd.read_csv('tags.csv')


In [3]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
ratings_crosstab=pd.pivot_table(data=ratings, values='rating', index='userId', columns='movieId')
ratings_crosstab

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,2.5,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# pick ratings given to the inputed movies
random_mov= 2
random_mov_ratings= ratings_crosstab[random_mov]
random_mov_ratings

userId
1      NaN
2      NaN
3      NaN
4      NaN
5      NaN
      ... 
606    NaN
607    NaN
608    2.0
609    NaN
610    NaN
Name: 2, Length: 610, dtype: float64

In [6]:
# Calc the correlations between the inputed movie and the rest
same_to_mov = ratings_crosstab.corrwith(random_mov_ratings)
same_to_mov

C:\Users\yusuf\anaconda3\lib\site-packages\numpy\lib\function_base.py:2634: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
C:\Users\yusuf\anaconda3\lib\site-packages\numpy\lib\function_base.py:2493: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


movieId
1         0.330978
2         1.000000
3         0.419564
4              NaN
5         0.562791
            ...   
193581         NaN
193583         NaN
193585         NaN
193587         NaN
193609         NaN
Length: 9724, dtype: float64

In [7]:
# drop missing values
corr = pd.DataFrame(same_to_mov, columns=['PearsonR'])
corr.dropna(inplace=True)
corr

,PearsonR
movieId,
1,0.330978
2,1.000000
3,0.419564
5,0.562791
6,0.163510
...,...
182823,-1.000000
184471,0.882498
185135,1.000000


In [8]:
#create a new df with rating_count
random_mov_count = pd.DataFrame(ratings.groupby('movieId')['rating'].mean())
random_mov_count['rating_count'] =ratings.groupby('movieId')['rating'].count()
random_mov_count

,rating,rating_count
movieId,,
1,3.920930,215
2,3.431818,110
3,3.259615,52
4,2.357143,7
5,3.071429,49
...,...,...
193581,4.000000,1
193583,3.500000,1
193585,3.500000,1


In [9]:
random_mov_count['rating_count'].value_counts()

1      3446
2      1298
3       800
4       530
5       382
       ... 
162       1
202       1
192       1
145       1
185       1
Name: rating_count, Length: 177, dtype: int64

In [10]:
# join corr coeffiecients with rating counts
corr_summary =corr.join(random_mov_count['rating_count'])
corr_summary.drop(random_mov, inplace=True) # drop the inputed movie itself

In [11]:
#filter movies with less than 50 rating_count and pick the top 10
top10 = corr_summary[corr_summary['rating_count']>=50].sort_values('PearsonR', ascending=False).head(10)

In [12]:
# merge with places to get the movie titles
top10 = top10.merge(movies, left_index=True, right_on="movieId")

In [13]:
top10

,PearsonR,rating_count,movieId,title,genres
2007,0.798372,56,2671,Notting Hill (1999),Comedy|Romance
5732,0.716712,52,30707,Million Dollar Baby (2004),Drama
3461,0.698576,55,4720,"Others, The (2001)",Drama|Horror|Mystery|Thriller
1124,0.687060,52,1466,Donnie Brasco (1997),Crime|Drama
7413,0.673058,57,80463,"Social Network, The (2010)",Drama
140,0.669709,54,168,First Knight (1995),Action|Drama|Romance
1135,0.659437,74,1485,Liar Liar (1997),Comedy
951,0.656153,59,1252,Chinatown (1974),Crime|Film-Noir|Mystery|Thriller
7466,0.656139,58,81845,"King's Speech, The (2010)",Drama
549,0.634676,65,653,Dragonheart (1996),Action|Adventure|Fantasy


In [14]:
def top_n_mov(movie_id, n):
    
    # pick ratings given to the inputed movies
    mov_ratings = ratings_crosstab[movie_id]
    
    # Calc the correlations between the inputed movie and the rest
    similar_to_mov = ratings_crosstab.corrwith(mov_ratings)
    
    # drop missing values
    corr_mov = pd.DataFrame(similar_to_mov, columns=['PearsonR'])
    corr_mov.dropna(inplace=True)
    
    #create a new df with rating_count
    mov_rating_count = pd.DataFrame(ratings.groupby('movieId')['rating'].mean())
    mov_rating_count['rating_count'] = ratings.groupby('movieId')['rating'].count()
    
    # join corr coeffiecients with rating counts
    mov_corr_summary = corr_mov.join(mov_rating_count['rating_count'])
    mov_corr_summary.drop(movie_id, inplace=True) # drop the inputed movie itself
    
    #filter movies with less than 50 rating_count and pick the top n
    top10 = mov_corr_summary[mov_corr_summary['rating_count']>=50].sort_values('PearsonR', ascending=False).head(n)
    
    # merge with places to get the movie titles
    top10 = top10.merge(movies, left_index=True, right_on="movieId")
    
    return list(top10["title"])
     

In [18]:
top_n_mov(79132,10)

['Full Monty, The (1997)',
 'Crow, The (1994)',
 'Grumpier Old Men (1995)',
 'Hook (1991)',
 'Desperado (1995)',
 'Interview with the Vampire: The Vampire Chronicles (1994)',
 'Super Size Me (2004)',
 'Maverick (1994)',
 'Leaving Las Vegas (1995)',
 'Animal House (1978)']

In [16]:
# to visualise al the columns in the dataframe
pd.pandas.set_option('display.max_rows', None)
movie_catalog= movies[['movieId','title']]
movie_catalog

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)
5,6,Heat (1995)
6,7,Sabrina (1995)
7,8,Tom and Huck (1995)
8,9,Sudden Death (1995)
9,10,GoldenEye (1995)
